In [1]:
import requests
import time
import pandas as pd
import os
from datetime import datetime


In [3]:
def request_api(url, **kwargs):
    payload={}
    headers = {
        "Content-Type": "application/json",
        "X-API-KEY": "..." # add your api key here
    }   
    return requests.request("GET", url, headers=headers, data=payload)


def get_daily_stats(dateday):
    #works nly after 02-23-2023 when it went live
    url="https://api.pollenmobile.io/explorer/daily-stats-day?date={dateday}".format(
        dateday=dateday
    )

    cnt=0
    check=False
    result = {}
    response = request_api(url)
    while (cnt<10) & (check==False):
        if response.status_code==200:
            result = response.json()
    
    return pd.DataFrame(result)

def get_all_stats():
    url="https://api.pollenmobile.io/explorer/daily-stats-all"

    cnt=0
    check=False
    result = {}
    response = request_api(url)
    while (cnt<10) & (check==False):
        if response.status_code==200:
            result = response.json()
    
    return pd.DataFrame(result)    



In [2]:
#adjust start and end date to get dataframe with network stats
start_date = '2022-02 -10'
end_date = '2022-12-01'

lst_failed = []
dct_out = {
    'dateday':[],
    'flower_share':[],
    'honeybee_share':[],
    'bumblebee_share':[],
    'PCNperPIC':[],
    'PICtotal':[],
    'PCNtotal':[],
    'honeybee_count':[],
    'bumblebee_count':[],
    'flower_count':[]
}

for dateday in pd.date_range(start=start_date,end=end_date):
    url = "https://api.pollenmobile.io/explorer/daily-stats-day?date={dateday}".format(
        dateday=datetime.strftime(dateday,'%Y-%m-%d')
    )


    response = request_api(url)
    json_out = response.json()
    if 'message' in json_out.keys():
        print('Failed run for {dateday}'.format(dateday=dateday))
        lst_failed.append(dateday)
        time.sleep(3)
    else:
        dct_out['dateday'].append(dateday)
        dct_out['flower_share'].append(float(json_out['flower_share']))
        if json_out['honeybee_share'] is None:
            dct_out['honeybee_share'].append(0)
        else:
            dct_out['honeybee_share'].append(float(json_out['honeybee_share']))
        if json_out['bumblebee_share'] is None:
            dct_out['bumblebee_share'].append(0)
        else:
            dct_out['bumblebee_share'].append(float(json_out['bumblebee_share']))
            
        dct_out['PCNperPIC'].append(float(json_out['PCNperPIC']))
        dct_out['PICtotal'].append(float(json_out['PICtotal']))
        dct_out['PCNtotal'].append(float(json_out['dailyPCN']))
        if json_out['honeybee_count'] is None:
            dct_out['honeybee_count'].append(0)
        else:
            dct_out['honeybee_count'].append(float(json_out['honeybee_count']))
        if json_out['bumblebee_count'] is None:
            dct_out['bumblebee_count'].append(0)
        else:
            dct_out['bumblebee_count'].append(float(json_out['bumblebee_count']))
        dct_out['flower_count'].append(float(json_out['flower_count']))
        time.sleep(2)

df_out=pd.DataFrame(dct_out)


In [50]:
df_out.to_csv('pollen_data_api.csv')